# Experimente zu Fallstudie 2: Repräsentanz Landesregierungen

## 1. Ansatz: Abgebildeter Text 

In [ ]:
from urllib.parse import urlparse
import json

import requests
from bs4 import BeautifulSoup
from bs4.element import Comment, Tag

In [ ]:
brandenburg_URL = "https://www.brandenburg.de/"

In [ ]:
html = requests.get(brandenburg_URL).text

In [ ]:
soup = BeautifulSoup(html)

In [ ]:
# https://stackoverflow.com/questions/1936466/how-to-scrape-only-visible-webpage-text-with-beautifulsoup

# Only get text that is visible on the website 
# Inlcudes ads, pointers to more info and so on 

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'head', 'title', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

def text_from_html(soup):
    texts = soup.findAll(string=True)
    visible_texts = filter(tag_visible, texts)  
    return [t.strip() for t in visible_texts if len(t) > 1]

In [ ]:
text = text_from_html(soup)

In [ ]:
text

### 1.2 Potentieller nächster Schritt: Filter
* all upper and short (DE, EN, PL)
* Ansprachen: "Sie sind" (start of text)
* Mehr Informationen
* Foto:
* weiterlesen
* alles nach "Links"

## 2. Ansatz: Strukturiertes Parsen

#### Nachteile von Ansatz 1
* Nicht nur Text, sondern auch Struktur von Website und Metadaten sind abgebildet (Impressum, Foto) 
* Die Links sind verloren gegangen
* Der Text ist unstrukturiert

#### Neuer Ansatz
* Für jeden Teilbereich der Seite eine spezialisierte Funktion schreiben, die die Daten strukturiert sammelt
* Links werden auch gesammelt und werden auch abgefragt

In [ ]:
# parse gallery (slider)
gallery = []

for h2 in soup.find_all("h2"):
    # check if h2 is part of gallery or "Aktuell" / "Themenschwerpunkte"
    if h2.string not in ["Aktuell", "Themenschwerpunkte"]:
        entry = {}
        entry["title"] = h2.string
        entry["description"] = h2.find_next_sibling("h3", {"class":"description"}).string
        entry["article_link"] = urlparse(h2.find_next_sibling("a").get("href"))
        gallery.append(entry)

In [ ]:
gallery

In [ ]:
# parse aktuell
aktuell = []
h2 = soup.find("h2", string="Aktuell")

for ul in h2.find_next("ul"):
    entry = {}
    if type(ul) is Tag:
        entry["Bereich"] = ul.find_next("h5").string
        entry["title"] = ul.find_next("h3").string
        teaser = ul.find_next("p")
        links = teaser.findChildren("a")
        if len(links) == 1:
            entry["article_link"] = urlparse(links[0].get("href"))
        # getText loses Fettung 
        entry["description"] = ul.find_next("p").getText()
        aktuell.append(entry)

In [ ]:
aktuell

In [ ]:
# parse Themenschwerpunkte
themenschwerpunkte = []
themenschwerpunkte_div = soup.find("div", {"class":"gallery-sp"})

for a in themenschwerpunkte_div.find_all("a"):
    entry = {}
    entry["article_link"] = urlparse(a.get("href"))
    entry["title"] = a.find("h4").string
    entry["description"] = a.find("p").getText()
    themenschwerpunkte.append(entry)

In [ ]:
themenschwerpunkte

In [ ]:
landesportal = {
    "gallery":gallery,
    "aktuell": aktuell,
    "themenschwerpunkte": themenschwerpunkte
}

In [ ]:
landesportal["gallery"][0]["article_link"].geturl()

### Follow links
* Links, die auf brandenburg.de enden
* Startseite wurde strukturiert geparsed, die verlinkten Seite sind noch messy

In [ ]:
def parse_pressemitteilung(html):
    soup = BeautifulSoup(html)
    text = [soup.title.string]
    p = soup.find_all("p")
    text.extend([entry.string for entry in p if entry.string is not None])
    return text

In [ ]:
for section, values in landesportal.items():
    for entry in values:
        if "article_link" in entry and entry["article_link"].netloc.endswith("brandenburg.de"):
            html = requests.get(entry["article_link"].geturl()).text
            text = parse_pressemitteilung(html)
            entry["text"] = text

In [ ]:
landesportal

### Ergebnis als JSON 

In [ ]:
from datetime import datetime

In [ ]:
datetime.now().strftime("%Y-%m-%d")

In [ ]:
with open(datetime.now().strftime("%Y-%m-%d") + "_landesportal.json", 'w') as json_out:
    json.dump(landesportal, json_out)